<img  align="right" width="250" src="recycling centre - pic.JPG" style= "float: right; margin: 0 50px 50px 50px;">

# Booking system project 

## Discovery of data insights to evaluate customer patterns and potential cost savings on a recycling centre

***


### Summary: In this project, booking system dataset from a recycling centre located in West London is analysed and main KPIs are calculated. 
### - Data cleaning and transformation was done as data quality issues were identified. 
### - All personal/sensitive data from datasets was encrypted or deleted for GDPR compliance.

<hr style="margin-bottom: 40px;">

### Main insights:


### Import of libraries

In [1]:
import numpy as np 
import pandas as pd
import random
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from matplotlib.patches import ConnectionPatch

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (14,8)

from datetime import datetime
import ipywidgets as widgets

### Import of all datasets 

In [2]:
#All booking details - Main dataset

Directory_main= r'C:\Users\Tiago\Desktop\Github\Professional_Portfolio\Booking system_dashboard\Recycling centre A - bookings.xlsx'
Main_dataset = pd.read_excel(Directory_main)

#Bookings daily capacity - dataset 

Directory_capacity = r'C:\Users\Tiago\Desktop\Github\Professional_Portfolio\Booking system_dashboard\Recycling centre A - max. capacity.xlsx'
Capacity_dataset = pd.read_excel(Directory_capacity)

#source: https://data.london.gov.uk/dataset/excel-mapping-template-for-london-boroughs-and-wards

Directory_wards = r'C:\Users\Tiago\Desktop\Github\Professional_Portfolio\Booking system_dashboard\Mapping-template-london-ward-map-2018.xlsx'
wards_boroughs_source = pd.read_excel(Directory_wards,usecols='B,C')

### Preview of datasets

In [3]:
Main_dataset.head(5)

,Booking created,Postcode,Ward,Type of vehicle,Site,Are you hiring a vehicle?,Registration plate,Batteries (car & household),Books,Bulky non-recyclables,...,Tyres (maximum 2),Wooden furniture (whole or dismantled),Asbestos (needs to be double wrapped),Garden wood (decking/fencing/shed),Kitchen and bathroom fixtures and fixings,Plasterboard,Soil and turf,Wooden flooring (including laminate),Booking date/time,ClosureReason
0,2023-02-18 16:02:04,HA9 9SL,Barnhill,Car,Recycling centre A (Car / Small van),No,KEaakzIUFU,Yes,No,Yes,...,No,No,No,No,No,No,No,No,2023-04-07 09:00:00,Complete
1,2023-03-03 15:56:33,HA0 3TJ,Northwick Park,Car,Recycling centre A (Car / Small van),Yes,HIREVEHICLE,No,No,No,...,No,No,No,No,No,No,No,No,2023-04-02 10:00:00,Complete
2,2023-03-14 21:05:24,W3 6TW,East Acton,Car,Recycling centre A (Car / Small van),No,LAbzaaDZT,Yes,No,No,...,No,No,No,No,No,No,No,No,2023-04-03 13:30:00,No show
3,2023-03-20 13:42:29,W10 5QG,Kensal Green,Car,Recycling centre A (Car / Small van),No,LOaabzbzRKX,No,Yes,Yes,...,No,Yes,No,No,No,No,No,No,2023-04-02 13:00:00,Cancelled by customer
4,2023-03-20 14:10:47,HA9 6DH,NaN,Car,Recycling centre A (Car / Small van),No,FHbzbzhbtUMF,Yes,No,No,...,No,Yes,No,No,No,No,No,No,2023-04-02 14:00:00,No show


In [4]:
Capacity_dataset.head()

,Date,Group,Total booked,Total spaces
0,01/04/2023,Recycling centre A (car / small van),145,400
1,01/04/2023,Recycling centre A (medium / large van),8,15
2,01/04/2023,Recycling centre A (pedestrian / bicycle),2,48
3,02/04/2023,Recycling centre A (car / small van),165,400
4,02/04/2023,Recycling centre A (medium / large van),14,15


In [5]:
wards_boroughs_source.head()

,Ward name,Borough name
0,Abbey,Barking and Dagenham
1,Alibon,Barking and Dagenham
2,Becontree,Barking and Dagenham
3,Chadwell Heath,Barking and Dagenham
4,Eastbrook,Barking and Dagenham


## Data Cleaning

### Columns headers terminology check and amendments done

In [6]:
Capacity_dataset_amended = (
    Capacity_dataset
    .rename(columns={'Group':'Transport type'
            ,'Date':'Booking_date'})
    .assign(**{'Transport type': lambda c: c['Transport type'].apply(lambda c: c[c.find('(')+1:c.find(')')])})
    .assign(
        Booking_dayofweek = lambda z: pd.to_datetime(z['Booking_date'], format='%d/%m/%Y').dt.day_name(),
        Booking_month = lambda a: pd.to_datetime(a['Booking_date'], format='%d/%m/%Y').dt.month_name()
    )
)
Capacity_dataset_amended

,Booking_date,Transport type,Total booked,Total spaces,Booking_dayofweek,Booking_month
0,01/04/2023,car / small van,145,400,Saturday,April
1,01/04/2023,medium / large van,8,15,Saturday,April
2,01/04/2023,pedestrian / bicycle,2,48,Saturday,April
3,02/04/2023,car / small van,165,400,Sunday,April
4,02/04/2023,medium / large van,14,15,Sunday,April
...,...,...,...,...,...,...
772,30/03/2024,medium / large van,15,15,Saturday,March
773,30/03/2024,pedestrian / bicycle,0,48,Saturday,March
774,31/03/2024,car / small van,155,400,Sunday,March
775,31/03/2024,medium / large van,15,15,Sunday,March


In [7]:
#check the names of the west London Boroughs
West_London_boroughs=['Brent','Ealing','Harrow','Hillingdon','Hounslow','Richmond upon Thames']

West_London_wards=(
    wards_boroughs_source
    .loc[wards_boroughs_source['Borough name'].isin(West_London_boroughs)].reset_index(drop=True)
)
West_London_wards.loc[West_London_wards['Ward name'].duplicated(keep=False)]

,Ward name,Borough name
1,Barnhill,Brent
14,Queensbury,Brent
58,Queensbury,Harrow
65,Barnhill,Hillingdon


In [8]:
#fixing the Ward name duplicates for Brent 

West_London_wards_amended= (
    West_London_wards
    .assign(
        **{
            'Ward name': lambda West_London_wards_: West_London_wards_['Ward name']
            .mask((West_London_wards_['Ward name']=='Barnhill') &(West_London_wards_['Borough name']=='Brent'),'Barnhill - Brent')
            .mask((West_London_wards_['Ward name']=='Queensbury') & (West_London_wards_['Borough name']=='Brent'),'Queensbury - Brent')
            }
        )
)

In [9]:
Main_dataset_amended = (
    Main_dataset
    .rename(
        columns={
            'Mattresses (from your home only)':'Mattresses',
            'Clothes and textiles (mixed)':'clothes and textiles',
            'Site':'Transport type',
            'Ward':'Ward name'
        }
    )
    .assign(
        Booking_date = lambda x: pd.to_datetime(x['Booking date/time']).dt.strftime('%d/%m/%Y'),
        Booking_time = lambda y: pd.to_datetime(y['Booking date/time']).dt.strftime('%H:%M')
    )
    .drop(columns=['Booking date/time'])
    .assign(
        **{'Booking created': pd.to_datetime(Main_dataset['Booking created']).dt.date},
        Booking_dayofweek = lambda z: pd.to_datetime(z['Booking_date'], format='%d/%m/%Y').dt.day_name(),
        Booking_month = lambda a: pd.to_datetime(a['Booking_date'], format='%d/%m/%Y').dt.month_name()
    )
    .assign(
        **{
        'Transport type': lambda b: b['Transport type'].apply(lambda b: b[b.find('(')+1:b.find(')')]).str.lower()
        }
    )
    .assign(
        **{
            'Ward name': lambda wards_replace:wards_replace['Ward name']
            .mask(
                (wards_replace['Ward name']=='Barnhill') & 
                (wards_replace['Postcode'].str.contains('HA9 9')), #all postcodes starting with HA9 or NW9 assigned to Barnhill Brent -https://www.doogal.co.uk/AdministrativeAreas?ward=E05013497
                 'Barnhill - Brent'
            )
            .mask(
                (wards_replace['Ward name']=='Queensbury') & 
                (wards_replace['Postcode'].str.contains('HA8|NW9|HA7')), #all postcodes starting with HA9 or NW9 assigned to Queensbury Brent -  Brenthttps://www.doogal.co.uk/AdministrativeAreas?ward=E05013508
                 'Queensbury - Brent'
            )
        }
    )
    .merge(
        West_London_wards_amended[['Ward name','Borough name']],
        on=['Ward name'],
        how='left'
    )   
    .assign(
        **{
            'Borough name': lambda other_wards: other_wards['Borough name']
            .mask(
                   other_wards['Ward name'].notnull() &
                   (other_wards['Borough name'].isnull()),
                   'Other Boroughs'
                )
        }
    )
    .assign(
        **{
            'Ward name': lambda empty_cell: empty_cell['Ward name']
               .mask(
                   empty_cell['Ward name'].isnull(),
                   'No info'
               )
        }
    )
    .assign(
        **{
            'Borough name': lambda empty_cells: empty_cells['Borough name']
               .mask(
                   empty_cells['Borough name'].isnull(),
                   'No info'
               )
        }
    )
)

Main_dataset_amended


,Booking created,Postcode,Ward name,Type of vehicle,Transport type,Are you hiring a vehicle?,Registration plate,Batteries (car & household),Books,Bulky non-recyclables,...,Kitchen and bathroom fixtures and fixings,Plasterboard,Soil and turf,Wooden flooring (including laminate),ClosureReason,Booking_date,Booking_time,Booking_dayofweek,Booking_month,Borough name
0,2023-02-18,HA9 9SL,Barnhill - Brent,Car,car / small van,No,KEaakzIUFU,Yes,No,Yes,...,No,No,No,No,Complete,07/04/2023,09:00,Friday,April,Brent
1,2023-03-03,HA0 3TJ,Northwick Park,Car,car / small van,Yes,HIREVEHICLE,No,No,No,...,No,No,No,No,Complete,02/04/2023,10:00,Sunday,April,Brent
2,2023-03-14,W3 6TW,East Acton,Car,car / small van,No,LAbzaaDZT,Yes,No,No,...,No,No,No,No,No show,03/04/2023,13:30,Monday,April,Ealing
3,2023-03-20,W10 5QG,Kensal Green,Car,car / small van,No,LOaabzbzRKX,No,Yes,Yes,...,No,No,No,No,Cancelled by customer,02/04/2023,13:00,Sunday,April,Brent
4,2023-03-20,HA9 6DH,No info,Car,car / small van,No,FHbzbzhbtUMF,Yes,No,No,...,No,No,No,No,No show,02/04/2023,14:00,Sunday,April,No info
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52463,2024-03-31,NW6 6SH,No info,Car,car / small van,No,LAaabzbzXUJ,No,No,No,...,Yes,No,No,No,Complete,31/03/2024,15:30,Sunday,March,No info
52464,2024-03-31,W10 2QS,No info,Car,car / small van,No,YHHty,No,No,No,...,No,No,No,No,Complete,31/03/2024,15:30,Sunday,March,No info
52465,2024-03-31,HA0 4RH,Wembley Central,Car,car / small van,No,AYbzbzkzIYHZ,No,No,No,...,No,No,No,No,Complete,31/03/2024,15:30,Sunday,March,Brent
52466,2024-03-31,NW6 6TH,Queens Park,Car,car / small van,No,KhbtWVH,No,No,No,...,No,No,No,No,Complete,31/03/2024,15:30,Sunday,March,Brent


In [10]:
#.merge(
        Capacity_dataset_amended[['Booking_date','Transport type','Total spaces']], 
                                     on=['Booking_date','Transport type'],
                                     how='left'
    #)

IndentationError: unexpected indent (600718513.py, line 2)

In [ ]:
app = dash.Dash(__name__)

#-----------------------------------------

Main_dataset_amended_details=Main_dataset_amended[
    [
        'Booking_date',
        'Booking_month',
        'Booking_dayofweek',
        'Borough name',
        'Type of vehicle',
        'Transport type',
        'Registration plate',
        'ClosureReason',
        'ClosureReason',
    ]
]

Capacity_dataset_amended

#--------------------------------------------
#App layout

app.layout = html.Div([

    html.H1("Booking system dashboard - Financial year 2023-2024", style={'text-align':'center'}),

    dcc.Dropdown(id="slct_month",
                 options=[
                     {"label":"April-23", "value": "April"},
                     {"label":"May-23", "value":"May"},
                     {"label":"Jun-23", "value":"June"},
                     {"label":"Jul-23", "value":"July"},
                     {"label":"Aug-23", "value":"August"},
                     {"label":"Sep-23", "value":"September"},
                     {"label":"Oct-23", "value":"October"},
                     {"label":"Nov-23", "value":"November"},
                     {"label":"Dec-23", "value":"December"},
                     {"label":"Jan-24", "value":"January"},
                     {"label":"Feb-24", "value":"February"},
                     {"label":"Mar-24", "value":"March"}],
                multi=False,
                value= "April",
                style = {'width':"40%"}
                 ),
    
    html.Div(id='output_container',children=[]),
    html.Br(),

    dcc.Graph(id='chart1',figure={})

])

#--------------------------------------------------------------------
#Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container',component_property='children'),
     Output(component_id='chart1',component_property='figure')],
    [Input(component_id='slct_month', component_property='value')]
)

def update_graphs(option_slctd): #atm, we only have one input, so only one argument
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    Booking_capacity = Capacity_dataset_amended.copy()
    Booking_capacity = Booking_capacity[Booking_capacity['Booking_month'] == option_slctd]

    # Ensure 'Booking_date' is in datetime format and sort by 'Booking_date'

    Booking_capacity['Booking_date'] = pd.to_datetime(Booking_capacity['Booking_date'],format='%d/%m/%Y', dayfirst=True)

    #Group total spaces and total bookings data by Booking_date

    dff_capacity_grouped = Booking_capacity.groupby('Booking_date').sum('Total spaces')
    dff_total_bookings = Booking_capacity.groupby('Booking_date').sum('Total booked')
    
    #Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces

    fig.add_trace(
        go.Bar(
            x=dff_total_bookings.index,
            y=dff_total_bookings['Total booked'],
            name='No. of bookings'
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=dff_capacity_grouped.index,
            y=dff_capacity_grouped['Total spaces'],
            name='Daily capacity',
            ),
        secondary_y=True
    )

    #Add figure title
    fig.update_layout(
        title_text="Number of bookings made vs bookings capacity"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Date")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Bookings capacity</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>No. of bookings</b>", secondary_y=True)
    fig.update_yaxes(range = [0,500])
    fig.update_xaxes(
        tickformat='%d',
        tickmode='array',
        tickvals=Booking_capacity['Booking_date']

    )
    fig.show()

    return container, fig
#--------------------------------------------------------------------

if __name__=='__main__':
    app.run_server(debug=True)



April
<class 'str'>


April
<class 'str'>


August
<class 'str'>


May
<class 'str'>


August
<class 'str'>
